<a href="https://colab.research.google.com/github/jayrom/gs_1_25/blob/main/GS_1_Sem_2025_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import time
import sys
import os
import joblib


## Conexão com Database


In [2]:
!pip install oracledb

In [3]:
import oracledb
from google.colab import userdata
db_user = userdata.get('DB_USER')
db_pass = userdata.get('DB_PASS')
dsn = 'oracle.fiap.com.br:1521/ORCL'

try:
  conn = oracledb.connect(user=db_user, password=db_pass, dsn=dsn)
  inst_cadastro = conn.cursor()
  inst_consulta = conn.cursor()
  conexao = True
  print("--- Conexão com a base de dados estabelecida")
except Exception as e:
  print(f"Erro ao conectar com a base de dados: {e}")
  conexao = False

--- Conexão com a base de dados estabelecida


In [4]:
import pandas as pd
import oracledb
from google.colab import userdata

conn = None
inst_cadastro = None
inst_consulta = None
conexao = False

def store_river_history(conn, inst_cadastro) -> None:
  print("Popular histórico do Rio\n")

  river_history_file = "document/data_inception/ITAICI_river_history.csv"
  river_history_data = None

  try:
    river_history_data = pd.read_csv(river_history_file)
    print(f"Arquivo '{river_history_file}' carregado com sucesso.")

  except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em '{river_history_file}'")
    return
  except Exception as e:
    print(f"ERRO: Coluna não encontrada. Verifique a Correspondência com o arquivo de dados '{e}'")
    return


  if river_history_data.empty:
    print("ERRO: Arquivo CSV está vazio.")
    return

  try:
    sql_insert = """
    INSERT INTO T_4E_047_ITAICI_RIVER_HISTORY (
      reading_id,
      reading_timestamp,
      reading_water_level,
      flag_flood,
      reading_calculated_water_pressure,
      flag_overcharged
    ) VALUES (SEQ_T_4E_047_ITAICI_RIVER_HISTORY.NEXTVAL, :1, :2, :3, :4, :5)
    """

    for index, row in river_history_data.iterrows():
      try:
        reading_timestamp = row['reading_timestamp']
        reading_water_level = float(row['reading_water_level'])
        flag_flood = int(row['flag_flood'])
        reading_calculated_water_pressure = float(row['reading_calculated_water_pressure'])
        flag_overcharged = int(row['flag_overcharged'])

        values = (
            reading_timestamp,
            reading_water_level,
            flag_flood,
            reading_calculated_water_pressure,
            flag_overcharged
        )

        inst_cadastro.execute(sql_insert, values)
        conn.commit()

        print(f"Inserindo registro: {values}")

      except Exception as e:
        print(f"Erro ao inserir registro {index}: {e}")

  except ValueError as ve:
        print(f"Erro de valor: {ve}") # Corrigido f-string
  except Exception as e: # Captura erros de conexão ou outros
        print(f"Erro na conexão com o DB ou durante a operação: {e}")
  else:
        print("Operação finalizada.")

## Carregamento de Dados Históricos

In [5]:
DATA_NORMALIZED_FILE = 'ITAICI_river_history.csv'
SENSOR_READINGS_FILE = "4E047_sensor_readings.csv"

PONTE_ID_SIMULACAO = 1
SIMULATION_INTERVAL_SECONDS = 2

PONTES_DATA = {
    1: {
        'equip_name': 'Ponte Principal',
        'equip_main_material': 'Concreto Armado',
        'equip_charge_rupture_limit': 350.0,
        'equip_safety_factor': 1.5
    },
    2: {
        'equip_name': 'Ponte Principal',
        'equip_main_material': 'Aço',
        'equip_charge_rupture_limit': 400.0,
        'equip_safety_factor': 1.8
    }
}

print("Dados da ponte carregados para a lógica de alerta")

Dados da ponte carregados para a lógica de alerta


## Preparação de Dados para ML

In [6]:
try:
    dados_treinamento = pd.read_csv(
        DATA_NORMALIZED_FILE,
        parse_dates=['reading_timestamp'],
        date_format='%Y-%m-%d %H:%M:%S'
    )
    print(f"Dados de treinamento carregados de '{DATA_NORMALIZED_FILE}' com sucesso.")
except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em '{DATA_NORMALIZED_FILE}'")
    sys.exit(1)
except Exception as e:
    print(f"Erro ao carregar '{DATA_NORMALIZED_FILE}': {e}")
    sys.exit(1)

features = ['reading_calculated_water_pressure']
target = 'flag_flood'

X = dados_treinamento[features]
y = dados_treinamento[target]

if len(X) < 20:
    print("Aviso: Poucos dados para treinamento. Considere adicionar mais entradas ao CSV")

Dados de treinamento carregados de 'ITAICI_river_history.csv' com sucesso.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Divisão de Dados para Treinamento e Teste

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dados divididos: Treino ({len(X_train)} amostras), Teste ({len(X_test)} amostras)")

Dados divididos: Treino (415 amostras), Teste (104 amostras)


## Treinamento utilizando Regressão Logística

In [9]:
ml_model = LogisticRegression(random_state=42)

print("Treinando Modelo...")
ml_model.fit(X_train, y_train)
print("Modelo Treinado com sucesso!")

#Avaliação do modelo

y_pred = ml_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"\nAcurácia do modelo nos dados de teste: {accuracy:.2f}")
print(f"\nRelatório de Classificação:\n", report)


Treinando Modelo...
Modelo Treinado com sucesso!

Acurácia do modelo nos dados de teste: 1.00

Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       101
           1       1.00      1.00      1.00         3

    accuracy                           1.00       104
   macro avg       1.00      1.00      1.00       104
weighted avg       1.00      1.00      1.00       104



### Exemplo de predição para teste

1.   Simulando dado de sensor (Nível de água)
2.   Valores Altos devem levar a uma previsão de Cheia (1)
3.   Valores Baixos devem levar a uma previsão sem risco (0)



In [10]:
exemplo_pressao_alta = pd.DataFrame({'reading_calculated_water_pressure': [300.0]})
exemplo_pressao_baixa = pd.DataFrame({'reading_calculated_water_pressure': [100.0]})

previsao_alta = ml_model.predict(exemplo_pressao_alta)[0]
prob_alta = ml_model.predict_proba(exemplo_pressao_alta)[0][1]

previsao_baixa = ml_model.predict(exemplo_pressao_baixa)[0]
prob_baixa = ml_model.predict_proba(exemplo_pressao_baixa)[0][1]

print(f"\nExemplo: Pressão da Água 300.0 kPa -> Previsão de Cheia: {previsao_alta} (Probabilidade: {prob_alta:.2f})")
print(f"\nExemplo: Pressão da Água 100.0 kPa -> Previsão de Cheia: {previsao_baixa} (Probabilidade: {prob_baixa:.2f})")


Exemplo: Pressão da Água 300.0 kPa -> Previsão de Cheia: 0 (Probabilidade: 0.00)

Exemplo: Pressão da Água 100.0 kPa -> Previsão de Cheia: 0 (Probabilidade: 0.00)


## Iplementação Lógica de Alerta


In [11]:
def gerar_alerta(id_ponte: int, pressao_agua_pilastra_kpa: float, ml_model_trained) -> dict:
    alerta_gerado = 0
    severidade_risco_engenharia = 0
    severidade_risco_ml = 0
    mensagem_alerta = "Condições Normais"

    if ml_model_trained is None:
        return {
            "alerta_gerado": 0,
            "severidade_risco_ml": 0,
            "mensagem_alerta": "Modelo de ML não carregado para alerta"
        }

    """### Lógica de Alerta de Engenharia"""
    ponte_info = PONTES_DATA.get(id_ponte)
    if ponte_info:
        limite_ruptura_kpa = ponte_info['equip_charge_rupture_limit']
        fator_seguranca = ponte_info['equip_safety_factor']
        limiar_critico_engenharia = limite_ruptura_kpa * fator_seguranca

        if pressao_agua_pilastra_kpa > limiar_critico_engenharia:
            alerta_gerado = 1
            severidade_risco_engenharia = 2
            mensagem_alerta = f"ALERTA CRÍTICO: Pressão da Água({pressao_agua_pilastra_kpa:.2f} kPa) EXCEDEU O LIMITE DE SEGURANÇA DA PONTE ({limiar_critico_engenharia:.2f} kPa)!"
        elif pressao_agua_pilastra_kpa >= (limiar_critico_engenharia * 0.8):
            alerta_gerado = 1
            severidade_risco_engenharia = 1
            if mensagem_alerta == "Condições Normais":
                mensagem_alerta = f"ALERTA MÉDIO: Pressão da Água({pressao_agua_pilastra_kpa:.2f} kPa) está próximo do limite de segurança da ponte ({limiar_critico_engenharia:.2f} kPa)."
    else:
        print(f"Aviso: Dados da Ponte {id_ponte} não encontrados em PONTES_DATA. Lógica de engenharia não aplicada para esta leitura.")


    """### Lógica de Alerta ML (baseada na previsão de cheia)"""

    input_for_ml = pd.DataFrame({'reading_calculated_water_pressure': [pressao_agua_pilastra_kpa]})

    previsao_ml = ml_model_trained.predict(input_for_ml)[0]
    probabilidade_ml = ml_model_trained.predict_proba(input_for_ml)[0][1]

    if previsao_ml == 1:
        alerta_gerado = 1
        if probabilidade_ml >= 0.7:
            severidade_risco_ml = 2
            if severidade_risco_engenharia < 2:
                if "ALERTA MÉDIO" in mensagem_alerta and severidade_risco_engenharia < 2:
                    mensagem_alerta = mensagem_alerta.replace("ALERTA MÉDIO", "ALERTA CRÍTICO (ML)")
                elif "Condições Normais" in mensagem_alerta or severidade_risco_engenharia < 1:
                    mensagem_alerta = f"ALERTA CRÍTICO (ML): Alta probabilidade de cheia ({probabilidade_ml:.2f}) baseada em padrões históricos."
        else:
            severidade_risco_ml = 1
            if severidade_risco_engenharia == 0 and "Condições Normais" in mensagem_alerta:
                 mensagem_alerta = f"ALERTA MÉDIO (ML): Risco de cheia detectado ({probabilidade_ml:.2f})."
            elif severidade_risco_engenharia < 1:
                severidade_risco_ml = max(severidade_risco_ml, 1)



    severidade_final = max(severidade_risco_engenharia, severidade_risco_ml)

    return {
        "alerta_gerado": alerta_gerado,
        "severidade_risco_ml": severidade_final,
        "mensagem_alerta": mensagem_alerta
    }

##Simulação de Leitura do Sensor Integrada

In [12]:
from logging import exception
def simulate_sensor_readings_from_csv_for_alerts(trained_model, id_ponte, sim_interval_seconds):
  print(f"\n--- Iniciando Simulação de Leitura do Sensor Integrada")
  print(f"Dados do sensor de '{SENSOR_READINGS_FILE}' serão utilizadas")
  print(f"Pressione Ctrl+C a qualquer momento para interromper a simulação.")

  try:
    sim_data_df = pd.read_csv(
        SENSOR_READINGS_FILE,
        parse_dates=['reading_timestamp'],
        date_format='%d-%b-%y'
    )
    sim_data_df = sim_data_df.sort_values(by='reading_timestamp')
    print(f"CSV de simulação carregado. Total de {len(sim_data_df)} leituras para simular")

  except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{SENSOR_READINGS_FILE}'")
    return
  except KeyError as e:
    print(f"Erro: Coluna '{e}' não encontrada no CSV de simulação. Esperado 'reading_pressure'. Verifique os nomes das colunas.")
    return
  except Exception as e:
    print(f"Erro ao carregar CSV de simulação: {e}")
    return

  for index, row in sim_data_df.iterrows():
    try:
      simulated_pressure_kpa = float(row['reading_pressure'])

      print(f"\nSimulando leitura {index + 1} (Timestamp: {row['reading_timestamp']}):")
      print(f"\nPressão lida pelo sensor: {simulated_pressure_kpa:.2f} kPa")

      alert_result = gerar_alerta(id_ponte, simulated_pressure_kpa, trained_model)

      print(f"  Status do Alerta: {alert_result['mensagem_alerta']}")
      print(f"  Severidade Geral (0=Baixa, 1=Média, 2=Crítica): {alert_result['severidade_risco_ml']}")

      time.sleep(sim_interval_seconds)
    except KeyboardInterrupt:
      print(f"\nSimulação interrompida pelo usuário.")
      break
    except Exception as e:
      print(f"Erro ao simular leitura {index}: {e}")
      continue
  print("\n--- Simulação de Leitura do Sensor Finalizada. ---")

if ml_model is not None and PONTES_DATA:
  simulate_sensor_readings_from_csv_for_alerts(ml_model, PONTE_ID_SIMULACAO, SIMULATION_INTERVAL_SECONDS)
else:
  print("--- Não foi possível iniciar a simulação: Modelo ML não treinado ou dados da ponte não carregados.")


--- Iniciando Simulação de Leitura do Sensor Integrada
Dados do sensor de '4E047_sensor_readings.csv' serão utilizadas
Pressione Ctrl+C a qualquer momento para interromper a simulação.
CSV de simulação carregado. Total de 65 leituras para simular

Simulando leitura 1 (Timestamp: 2025-06-03 00:00:00):

Pressão lida pelo sensor: 200.34 kPa
  Status do Alerta: Condições Normais
  Severidade Geral (0=Baixa, 1=Média, 2=Crítica): 0

Simulando leitura 2 (Timestamp: 2025-06-04 00:00:00):

Pressão lida pelo sensor: 197.35 kPa
  Status do Alerta: Condições Normais
  Severidade Geral (0=Baixa, 1=Média, 2=Crítica): 0

Simulando leitura 3 (Timestamp: 2025-06-05 00:00:00):

Pressão lida pelo sensor: 198.05 kPa
  Status do Alerta: Condições Normais
  Severidade Geral (0=Baixa, 1=Média, 2=Crítica): 0

Simulando leitura 4 (Timestamp: 2025-06-06 00:00:00):

Pressão lida pelo sensor: 202.98 kPa
  Status do Alerta: Condições Normais
  Severidade Geral (0=Baixa, 1=Média, 2=Crítica): 0

Simulando leitura 5

## Lógica de treinamento - Streamlit

In [13]:
# treinar_e_salvar_modelo.py

# --- IMPORTS NECESSÁRIOS ---
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib
import sys
# ---------------------------

print("--- Iniciando o treinamento do modelo ---")

DATA_NORMALIZED_FILE = 'ITAICI_river_history.csv'

# Carregar os dados de treinamento
try:
    dados_treinamento = pd.read_csv(
        DATA_NORMALIZED_FILE,
        parse_dates=['reading_timestamp'],
        date_format='%Y-%m-%d %H:%M:%S'
    )
    print(f"Dados de treinamento carregados de '{DATA_NORMALIZED_FILE}' com sucesso.")
except FileNotFoundError:
    print(f"ERRO: Arquivo '{DATA_NORMALIZED_FILE}' não encontrado. Certifique-se de que ele está na mesma pasta.")
    sys.exit(1)

# Preparação dos dados
features = ['reading_calculated_water_pressure']
target = 'flag_flood'
X = dados_treinamento[features]
y = dados_treinamento[target]

# Divisão de dados (opcional para o script final, mas bom para consistência)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo
ml_model = LogisticRegression(random_state=42)
print("Treinando o modelo de Regressão Logística...")
ml_model.fit(X_train, y_train)
print("Modelo treinado com sucesso!")

# Salvar o modelo treinado em um arquivo
NOME_ARQUIVO_MODELO = 'modelo_cheia.joblib'
joblib.dump(ml_model, NOME_ARQUIVO_MODELO)
print(f"Modelo salvo com sucesso no arquivo: '{NOME_ARQUIVO_MODELO}'")
print("--- Processo de treinamento finalizado ---")

--- Iniciando o treinamento do modelo ---
Dados de treinamento carregados de 'ITAICI_river_history.csv' com sucesso.
Treinando o modelo de Regressão Logística...
Modelo treinado com sucesso!
Modelo salvo com sucesso no arquivo: 'modelo_cheia.joblib'
--- Processo de treinamento finalizado ---


## Criando a aplicação - Streamlit

In [14]:
!pip install streamlit pyngrok -q

In [15]:
%%writefile app.py
# app_monitoramento.py
import streamlit as st
import pandas as pd
import time
import joblib

st.set_page_config(page_title="Monitoramento de Ponte", layout="wide")
st.title("Meu Aplicativo de Alerta")
st.write("Este é o meu app rodando a partir de um arquivo .py criado magicamente!")


# --- Configurações Iniciais e Carregamento de Dados ---

# Título e configuração da página
st.title("🚨 Dashboard de Monitoramento de Pressão na Ponte")

# Dicionário de dados da ponte (copiado do seu script)
PONTES_DATA = {
    1: {
        'equip_name': 'Ponte Principal',
        'equip_main_material': 'Concreto Armado',
        'equip_charge_rupture_limit': 350.0,
        'equip_safety_factor': 1.5
    },
    2: {
        'equip_name': 'Ponte Secundária',
        'equip_main_material': 'Aço',
        'equip_charge_rupture_limit': 400.0,
        'equip_safety_factor': 1.8
    }
}
PONTE_ID_SIMULACAO = 1
SIMULATION_INTERVAL_SECONDS = 2
SENSOR_READINGS_FILE = "4E047_sensor_readings.csv"

# Função para carregar o modelo e os dados (com cache para performance)
@st.cache_resource
def carregar_modelo():
    try:
        modelo = joblib.load('modelo_cheia.joblib')
        return modelo
    except FileNotFoundError:
        st.error("Arquivo 'modelo_cheia.joblib' não encontrado. Execute o script 'treinar_e_salvar_modelo.py' primeiro.")
        return None

@st.cache_data
def carregar_dados_simulacao():
    try:
        sim_data_df = pd.read_csv(
            SENSOR_READINGS_FILE,
            parse_dates=['reading_timestamp'],
            date_format='%d-%b-%y'
        ).sort_values(by='reading_timestamp')
        return sim_data_df
    except FileNotFoundError:
        st.error(f"Arquivo de simulação '{SENSOR_READINGS_FILE}' não encontrado.")
        return None

# Carrega os recursos
ml_model = carregar_modelo()
sim_data_df = carregar_dados_simulacao()

# --- Lógica de Alerta (copiada e adaptada do seu script) ---
# Esta função é a mesma que você criou, não precisa de alterações.
def gerar_alerta(id_ponte: int, pressao_agua_pilastra_kpa: float, ml_model_trained) -> dict:
    alerta_gerado = 0
    severidade_risco_engenharia = 0
    severidade_risco_ml = 0
    mensagem_alerta = "Condições Normais"
    probabilidade_ml = 0.0

    if ml_model_trained is None:
        return {"alerta_gerado": 0, "severidade_risco_ml": 0, "mensagem_alerta": "Modelo de ML não carregado", "probabilidade_ml": 0}

    ponte_info = PONTES_DATA.get(id_ponte)
    if ponte_info:
        limite_ruptura_kpa = ponte_info['equip_charge_rupture_limit']
        fator_seguranca = ponte_info['equip_safety_factor']
        limiar_critico_engenharia = limite_ruptura_kpa * fator_seguranca

        if pressao_agua_pilastra_kpa > limiar_critico_engenharia:
            alerta_gerado = 1
            severidade_risco_engenharia = 2
            mensagem_alerta = f"ALERTA CRÍTICO: Pressão da Água({pressao_agua_pilastra_kpa:.2f} kPa) EXCEDEU O LIMITE DE SEGURANÇA DA PONTE ({limiar_critico_engenharia:.2f} kPa)!"
        elif pressao_agua_pilastra_kpa >= (limiar_critico_engenharia * 0.8):
            alerta_gerado = 1
            severidade_risco_engenharia = 1
            if mensagem_alerta == "Condições Normais":
                mensagem_alerta = f"ALERTA MÉDIO: Pressão da Água({pressao_agua_pilastra_kpa:.2f} kPa) está próximo do limite de segurança da ponte ({limiar_critico_engenharia:.2f} kPa)."

    input_for_ml = pd.DataFrame({'reading_calculated_water_pressure': [pressao_agua_pilastra_kpa]})
    previsao_ml = ml_model_trained.predict(input_for_ml)[0]
    probabilidade_ml = ml_model_trained.predict_proba(input_for_ml)[0][1]

    if previsao_ml == 1:
        alerta_gerado = 1
        if probabilidade_ml >= 0.7:
            severidade_risco_ml = 2
            if severidade_risco_engenharia < 2:
                mensagem_alerta = f"ALERTA CRÍTICO (ML): Alta probabilidade de cheia ({probabilidade_ml:.2f}) detectada."
        else:
            severidade_risco_ml = 1
            if severidade_risco_engenharia == 0:
                mensagem_alerta = f"ALERTA MÉDIO (ML): Risco de cheia detectado ({probabilidade_ml:.2f})."

    severidade_final = max(severidade_risco_engenharia, severidade_risco_ml)

    return {"alerta_gerado": alerta_gerado, "severidade_final": severidade_final, "mensagem_alerta": mensagem_alerta, "probabilidade_ml": probabilidade_ml}


# --- Interface do Usuário e Lógica de Simulação ---

if ml_model and sim_data_df is not None:
    st.sidebar.header("Controles da Simulação")
    # Botão para iniciar a simulação
    if st.sidebar.button('Iniciar Monitoramento'):

        # Placeholders para os elementos dinâmicos do dashboard
        status_placeholder = st.empty()
        cols = st.columns(3)
        kpi1_placeholder = cols[0]
        kpi2_placeholder = cols[1]
        kpi3_placeholder = cols[2]
        chart_placeholder = st.empty()

        # Dataframe para o gráfico em tempo real
        history_df = pd.DataFrame(columns=['Leitura', 'Pressão (kPa)'])

        for index, row in sim_data_df.iterrows():
            simulated_pressure_kpa = float(row['reading_pressure'])

            # Gera o alerta com base na leitura atual
            alert_result = gerar_alerta(PONTE_ID_SIMULACAO, simulated_pressure_kpa, ml_model)

            # ===== AQUI ESTÁ A LÓGICA DO ALERTA POP-UP =====
            if alert_result['severidade_final'] == 2: # Alerta Crítico
                st.toast(f"URGENTE: {alert_result['mensagem_alerta']}", icon='🚨')
            elif alert_result['severidade_final'] == 1: # Alerta Médio
                st.toast(alert_result['mensagem_alerta'], icon='⚠️')
            # =======================================================

            # Atualiza o painel de status fixo
            with status_placeholder.container():
                if alert_result['severidade_final'] == 2:
                    st.error(f"**Status Atual:** {alert_result['mensagem_alerta']}", icon="🚨")
                elif alert_result['severidade_final'] == 1:
                    st.warning(f"**Status Atual:** {alert_result['mensagem_alerta']}", icon="⚠️")
                else:
                    st.success(f"**Status Atual:** {alert_result['mensagem_alerta']}", icon="✅")

            # Atualiza os KPIs
            limite_seguranca = PONTES_DATA[PONTE_ID_SIMULACAO]['equip_charge_rupture_limit'] * PONTES_DATA[PONTE_ID_SIMULACAO]['equip_safety_factor']
            kpi1_placeholder.metric(label="Pressão Atual", value=f"{simulated_pressure_kpa:.2f} kPa")
            kpi2_placeholder.metric(label="Limite de Segurança", value=f"{limite_seguranca:.2f} kPa")
            kpi3_placeholder.metric(label="Probabilidade de Cheia (ML)", value=f"{alert_result['probabilidade_ml']:.2%}")

            # Atualiza o gráfico
            new_row = pd.DataFrame({'Leitura': [row['reading_timestamp']], 'Pressão (kPa)': [simulated_pressure_kpa]})
            history_df = pd.concat([history_df, new_row], ignore_index=True)
            history_df['Leitura'] = pd.to_datetime(history_df['Leitura'])
            with chart_placeholder.container():
                st.line_chart(history_df.rename(columns={'Leitura':'index'}).set_index('index'))

            # Aguarda para a próxima leitura
            time.sleep(SIMULATION_INTERVAL_SECONDS)

        st.info("Simulação finalizada.")
else:
    st.warning("Não foi possível carregar o modelo ou os dados de simulação. Verifique os arquivos.")

Overwriting app.py


In [16]:
!ls -l


!cat app.py


!pip install streamlit pyngrok -q

from pyngrok import ngrok
ngrok.set_auth_token("2y9y9ei5ebOM4X9gvkoQAJsv7ug_385EQ31SPXmxA3uLLmJCh")


!nohup streamlit run app.py &


import time
time.sleep(5)


public_url = ngrok.connect(addr="8501")
print("Sua aplicação Streamlit está disponível em:", public_url)

total 52
-rw-r--r-- 1 root root  1322 Jun  6 23:55 4E047_sensor_readings.csv
-rw-r--r-- 1 root root  7504 Jun  7 02:00 app.py
drwx------ 6 root root  4096 Jun  6 23:03 drive
-rw-r--r-- 1 root root 14376 Jun  6 23:00 ITAICI_river_history.csv
-rw-r--r-- 1 root root  1167 Jun  7 02:00 modelo_cheia.joblib
-rw------- 1 root root  8407 Jun  7 02:00 nohup.out
drwxr-xr-x 1 root root  4096 Jun  5 13:38 sample_data
# app_monitoramento.py
import streamlit as st
import pandas as pd
import time
import joblib

st.set_page_config(page_title="Monitoramento de Ponte", layout="wide")
st.title("Meu Aplicativo de Alerta")
st.write("Este é o meu app rodando a partir de um arquivo .py criado magicamente!")
 

# --- Configurações Iniciais e Carregamento de Dados ---

# Título e configuração da página
st.title("🚨 Dashboard de Monitoramento de Pressão na Ponte")

# Dicionário de dados da ponte (copiado do seu script)
PONTES_DATA = {
    1: {
        'equip_name': 'Ponte Principal',
        'equip_main_material